In [ ]:
# problem when different schemas
from sqlalchemy import create_engine
import pandas as pd
def get_engine(user = 'postgres', password = 'dummy', host = 'localhost', port = '5432', dbname = 'sqlrag'):

    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

    return create_engine(connection_string)

# get engine
engine = get_engine()

In [ ]:
# create and fill employee table
data = {
    "employee_id": [1, 2, 3, 4],
    "employee_name": ["Employee_1", "Employee_2", "Employee_3", "Employee_4"],
}
df = pd.DataFrame(data)
schema = "sch_employee"
table_name = 'employee'
df.to_sql(table_name, engine, if_exists='replace', index=False, schema=schema)  # Use 'fail' or 'append' as needed

In [ ]:
# create and fill worked-hours table
data = {
    "employee_id": [1, 2, 3, 4],
    "worked_hours": [1, 22, 333, 4444],
}
df = pd.DataFrame(data)
table_name = 'hours'
df.to_sql(table_name, engine, if_exists='replace', index=False, schema=schema)

In [1]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_community.utilities import SQLDatabase
import os
import logging
logging.basicConfig(level=logging.ERROR)
llm = AzureChatOpenAI(azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), api_key=os.getenv("AZURE_OPENAI_API_KEY")
                        , azure_deployment=os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT"), model="gpt-35-turbo"
                        , api_version=os.getenv("OPENAI_API_VERSION"), openai_api_type="azure")

def get_db(src = "langchain", user = 'postgres', password = 'dummy', host = 'localhost', port = '5432', dbname = 'sqlrag'):

    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

    if src == "langchain":
        return SQLDatabase.from_uri(connection_string, schema = "sch_employee")
    
    print(f"Please provide proper src, not '{src}'")

db = get_db()

from langchain.globals import set_debug
set_debug(False)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

In [2]:
query = "How many horus were worked by which employee?"
query = "Which employee worked most hours? Just give me teh result."
agent_executor.invoke(query)

{'input': 'Which employee worked most hours? Just give me teh result.',
 'output': 'The employee with ID 4 worked the most hours, with a total of 4444 hours.'}

In [ ]:
# problems here

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
import logging
logging.basicConfig(level=logging.DEBUG)


engine = get_engine()
include_tables=["employee", "hours"]
schema = "sch_employee"
sql_database = SQLDatabase(engine=engine, include_tables=include_tables, schema=schema)
query_engine = NLSQLTableQueryEngine(sql_database=sql_database, tables = include_tables, verbose=True)

query = "Give ma a table of all employees and worked hours."
r = query_engine.query(query)
print(r)


In [ ]:
query = "Give ma a table of all employees and worked hours."
query = "Which tables from which schemas do you use?"
r = query_engine.query(query)
print(r)